# Inflation Index Example

This notebook demonstrates how to work with economic scenario generator data, specifically inflation indices.

Key concepts covered:
- Loading inflation index data from CSV files
- Creating ProteusVariable from economic scenario data
- Upsampling data to match simulation requirements
- Visualizing inflation index distributions

## Setup and Configuration

First, we'll import the necessary modules and configure the number of simulations.

In [21]:
from pal import config
from pal.variables import ProteusVariable

# Set the number of simulations
n_sims = 100000
config.n_sims = n_sims

print(f"Configured for {n_sims:,} simulations")

Configured for 100,000 simulations


## Load Inflation Index Data

We'll load inflation index data from a CSV file generated by an economic scenario generator.
The data contains time-series projections of inflation indices.

In [22]:
# Load inflation index from CSV
# The CSV contains columns: Time (dimension) and Index (values)
import os

# Check if we are in the examples directory or project root
csv_path = "data/proteus_scenario_generator/Economics_USD_Inflation_Inflation Index.csv"
if not os.path.exists(csv_path):
    csv_path = "../" + csv_path

inflation_index = ProteusVariable.from_csv(
    csv_path,
    "Time",  # Dimension column
    "Index",  # Value column
)

print(f"Loaded inflation index with dimensions: {inflation_index.dimensions}")
print(f"Number of time periods: {len(inflation_index.values)}")

Loaded inflation index with dimensions: ['Time']
Number of time periods: 11


## Upsample to Match Simulation Count

The economic scenario generator may produce fewer scenarios than needed for our simulation.
We'll upsample the data to match our required number of simulations.

In [23]:
# Upsample the inflation index to the correct number of simulations
upsampled_inflation_index = inflation_index.upsample(n_sims)

print(f"Upsampled inflation index to {n_sims:,} simulations")

Upsampled inflation index to 100,000 simulations


## Visualize Inflation Index Distribution

Let's visualize the cumulative distribution function (CDF) of the upsampled inflation index.
This shows the probability distribution of inflation outcomes across all simulations.

In [24]:
# Display the CDF of the upsampled inflation index
upsampled_inflation_index.show_cdf()

## Analysis and Insights

Let's extract key statistics from the inflation index data at different time horizons.

Note: In inflation modeling, time period 0 is typically the base period where all scenarios start at index 1.0.
The interesting variation appears in future time periods, showing how inflation might evolve under different economic scenarios.

In [25]:
import numpy as np

# Get statistics for different time periods
if upsampled_inflation_index.values:
    # Get available time periods
    time_periods = sorted(upsampled_inflation_index.values.keys())
    print(f"Available time periods: {time_periods[:10]}...")
    print("\nNote: The index shows CUMULATIVE inflation from the base period")
    print("(time 0)\n")

    # Skip time 0 (base period) and look at later periods for meaningful variation
    analysis_periods = [1, 5, 10] if len(time_periods) > 10 else [1]

    for period in analysis_periods:
        if str(period) in upsampled_inflation_index.values:
            period_data = upsampled_inflation_index.values[str(period)]
            mean_index = np.mean(period_data)

            print("\n" + "="*60)
            print(f"Time Period {period} (Year {period}):")
            print("="*60)

            # Basic statistics
            print("Index Statistics:")
            print(f"  Mean Index: {mean_index:.4f}")
            print(f"  Std Dev: {np.std(period_data):.4f}")
            print(f"  Min: {np.min(period_data):.4f}")
            print(f"  Max: {np.max(period_data):.4f}")

            # Inflation calculations
            cumulative_inflation = (mean_index - 1.0) * 100
            if period > 0:
                annualized_inflation = ((mean_index ** (1/period)) - 1) * 100
            else:
                annualized_inflation = 0

            print("\nInflation Metrics:")
            print(f"  Cumulative inflation from base: {cumulative_inflation:.2f}%")
            print(f"  Annualized inflation rate: {annualized_inflation:.2f}%")

            # Percentiles
            percentiles = [5, 25, 50, 75, 95]
            print("\nPercentiles (cumulative inflation from base):")
            for p in percentiles:
                value = np.percentile(period_data, p)
                pct_inflation = (value - 1.0) * 100
                print(f"  {p:3d}th percentile: {pct_inflation:6.2f}%")

print("\n" + "="*60)
print("Key Insights:")
print("- The index represents cumulative price levels, not annual rates")
print("- An index of 1.17 at year 5 means prices are 17% higher than base")
print("- This represents ~3.2% annualized inflation, which is reasonable")
print("- The variation shows different economic scenarios for stress testing")

Available time periods: ['0', '1', '10', '2', '3', '4', '5', '6', '7', '8']...

Note: The index shows CUMULATIVE inflation from the base period (time 0)


Time Period 1 (Year 1):
Index Statistics:
  Mean Index: 1.0335
  Std Dev: 0.0171
  Min: 0.9831
  Max: 1.2030

Inflation Metrics:
  Cumulative inflation from base: 3.35%
  Annualized inflation rate: 3.35%

Percentiles (cumulative inflation from base):
    5th percentile:   0.90%
   25th percentile:   2.26%
   50th percentile:   3.26%
   75th percentile:   4.27%
   95th percentile:   5.96%

Time Period 5 (Year 5):
Index Statistics:
  Mean Index: 1.1713
  Std Dev: 0.0858
  Min: 0.9626
  Max: 1.5888

Inflation Metrics:
  Cumulative inflation from base: 17.13%
  Annualized inflation rate: 3.21%

Percentiles (cumulative inflation from base):
    5th percentile:   5.29%
   25th percentile:  11.40%
   50th percentile:  16.20%
   75th percentile:  21.83%
   95th percentile:  33.31%

Time Period 10 (Year 10):
Index Statistics:
  Mean Index: 1.

## Summary

This example demonstrated:
1. Loading economic scenario data from CSV files
2. Creating ProteusVariable objects from time-series data
3. Upsampling scenarios to match simulation requirements
4. Visualizing and analyzing inflation index distributions

The inflation index data can be used in actuarial models to:
- Project future claim costs with inflation
- Adjust reserves for inflation expectations
- Stress test portfolios under different inflation scenarios